# WA5FRF Chirp Experiment Analysis Script
Stephen A. Cerwin WA5FRF

Nathaniel A. Frissell W2NAF

September 7, 2023

September 6, 2023

Nathaniel,
Here are five .wav files. I included the callsigns of the two stations, the band, date, UTC time, and sample rate in the first four filenames. The first one is the test waveform and the next three are on-the-air transmissions. The WA5FRF-N5DUP files are the ones I used in the papers at your workshop. The KJ5MA waveform has not been analyzed. The last one voicetx1.wav is the waveform from the IC-7610\VoiceTx\ folder on the SD card in the Icom 7610. I recorded it using the normal Tx voice memory recording method and feeding the waveform from the computer through a transformer isolated input circuit to mic input of the radio. It plays from the T1 button (hence the tx1 in the filename) in the VOICE screen selection brought up with the MENU button on the bottom of the radio next to the SD card slot. 

~Steve Cerwin

## Load in Libraries and Define Functions

In [ ]:
import os

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

from IPython.display import Audio, display

from scipy import signal
from scipy.io import wavfile

import pandas as pd

%matplotlib inline

In [ ]:
# Sets default style and font parameters for the plots we are making.
mpl.rcParams['font.size']      = 16
mpl.rcParams['font.weight']    = 'bold'
mpl.rcParams['axes.grid']      = True
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['figure.figsize'] = (12,9)
mpl.rcParams['axes.xmargin']   = 0
mpl.rcParams['axes.ymargin']   = 0.1

In [ ]:
def play(df, column='x', autoplay=False,tlim=None):
    """
    Play a signal (WAV file) stored in a DataFrame.
    
    df:      DataFrame containing signal with time series index.
    column:  Name of column to play.
    autoplay: Immediately play the signal if True  
    """
    if tlim is not None:
        tf = np.logical_and(df.index >= tlim[0],
                            df.index <  tlim[1])
        df = df[tf].copy()
    
    # Get signal.
    x    = df[column]
    
    # Compute sampling period and sampling frequency.
    tvec = df.index
    Ts   = tvec[1] - tvec[0]
    fs   = 1/Ts
    
    display(Audio(x, rate=fs, autoplay=autoplay))# Enable/disable playing of sounds

In [ ]:
def adjust_axes(ax_0,ax_1):
    """
    Force geospace environment axes to line up with histogram
    axes even though it doesn't have a color bar.
    """
    ax_0_pos    = list(ax_0.get_position().bounds)
    ax_1_pos    = list(ax_1.get_position().bounds)
    ax_0_pos[2] = ax_1_pos[2]
    ax_0.set_position(ax_0_pos)

In [ ]:
def plot_sig(df,column='x',title=None,tlim=None,flim=None,figsize=(15,10),
            waveform=True, specgram=True):
    """
    Plot signal and spectrogram.
    
    df:      DataFrame containing signal with time series index.
    column:  Name of column to plot.
    fs:      sampling frequency
    title:   Overall title of plot
    tlim:    Time limits of waveform and spectrogram plots
    flim:    Frequency Limits of spectrogram and FFT plots
    figsize: Size of figure
    
    waveform: If True, plot waveform.
    specgram: If True, plot spectrogram.
    
    """
    # Get signal.
    x    = df[column]
    
    # Compute sampling period and sampling frequency.
    tvec = df.index
    Ts   = tvec[1] - tvec[0]
    fs   = 1/Ts
    
    # Set default time limits.
    if tlim is None:
        tlim = (0,np.max(tvec))
    
    fig = plt.figure(figsize=figsize) # Create figure.
    
    ### Plot the time domain waveform.
    axs = []
    if waveform:
        ax  = fig.add_subplot(2,1,1)
        axs.append(ax)
        ax.plot(tvec,x)
        ax.set_xlabel('t [sec]')
        ax.set_ylabel('x(t)')
        ax.set_xlim(tlim)

    ### Plot the spectrogram
    if specgram:
        ax  = fig.add_subplot(2,1,2)
        axs.append(ax)
        nperseg   = int(fs)           # 1 Hz resolution (df = fs/nperseg)
        noverlap  = int(0.75*nperseg) # 75% Overlap of Windows
        f, t, Sxx = signal.spectrogram(x, fs,window='hann',nperseg=nperseg,noverlap=noverlap)
        mpbl      = ax.pcolormesh(t, f, 10*np.log10(Sxx))
        ax.set_ylim(flim)
        ax.set_ylabel('Frequency [Hz]')
        ax.set_xlabel('Time [sec]')
        ax.set_xlim(tlim)
        plt.colorbar(mpbl,label='PSD [dB]',aspect=10,pad=0.02)
    
    if title is not None:
        axs[0].set_title(title)
        
    plt.tight_layout()
    
    if waveform and specgram:
        adjust_axes(axs[0],axs[1])
        
    plt.show()

In [ ]:
def plot_filter_response(sos,fs,Wn=None,db_lim=(-40,1),flim=None,figsize=(10,6)):
    """
    Plots the magnitude and phase response of a filter.
    
    sos:    second-order sections ('sos') array
    fs:     sample rate
    Wn:     cutoff frequency(ies)
    db_lim: ylimits of magnitude response plot
    flim:   frequency limits of plots
    """
    if Wn is not None:
        # Make sure Wn is an iterable.
        Wn = np.array(Wn)
        if Wn.shape == ():
            Wn.shape = (1,)
    
    f, h  = signal.sosfreqz(sos, worN=fs, fs=fs)
    
    fig = plt.figure(figsize=figsize)
    
    # Plot time domain waveform
    plt.subplot(211)
    plt.plot(f, 20 * np.log10(abs(h)))
    # plt.xscale('log')
    plt.title('Filter Frequency Response')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Amplitude [dB]')
    plt.grid(which='both', axis='both')
    if Wn is not None:
        for cf in Wn:
            plt.axvline(cf, color='green') # cutoff frequency
    plt.xlim(flim)
    plt.ylim(db_lim)

    # Plot spectrogram
    plt.subplot(212)
    plt.plot(f, np.unwrap(np.angle(h)))
    # plt.xscale('log')
    plt.title('Filter Phase Response')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Phase [rad]')
    plt.grid(which='both', axis='both')
    if Wn is not None:
        for cf in Wn:
            plt.axvline(cf, color='green') # cutoff frequency
    plt.xlim(flim)

    plt.tight_layout()
    plt.show()

In [ ]:
def load_wav(fname,normalize=True):
    """    
    Loads a WAV file and returns a DataFrame with the signal
    and the sampling frequency.
    
    Input:
        fname:     Filename of WAV file.
        normalize: If True, normalize signal and convert to floating point.
        
    Returns:
        df: DataFrame containing signal
        fs: Sampling frequency [samples/sec]
    """
    # Load WAV file.
    fs,x0 = wavfile.read(fname)
    
    if normalize:
        x0   = x0/float(np.max(np.abs(x0)))

    # Compute time vector.
    N  = len(x0)             # Number of samples in signal
    k  = np.arange(len(x0))  # Integer time vector
    Ts = 1/fs                # Sampling Period
    t  = k*Ts
    
    df = pd.DataFrame({'time':t,'x':x0})
    df = df.set_index('time')
    
    return df, fs     

## Load and Plot WAV File

In [ ]:
# fname = "WA5FRF_seqp-test-48000SPS.wav"
# fname = "WA5FRF-KJ5MA-75m-20230725-1237z-16000SPS.wav"
# fname = "WA5FRF-N5DUP-60m-20230226-2104z-8000SPS.wav"
fname  = os.path.join('data','WA5FRF-N5DUP-75m-20230224-1410z-44100SPS.wav')
x0,fs  = load_wav(fname)

In [ ]:
flim   = (0,4e3) # Set frequency limits for plotting purposes.
plot_sig(x0,title=fname,flim=flim)

In [ ]:
# Play Wav File
play(x0)

## Select Time Period of Interest

In [ ]:
tlim  = (7.3,7.6)

In [ ]:
plot_sig(x0,title=fname,flim=flim,tlim=tlim)

In [ ]:
play(x0,tlim=tlim)

# START CODING HERE THE NEXT TIME

# Squaring the Signal

In [ ]:
m2 = m1**2



fig = plt.figure(figsize=(10,8)) # Create figure.

### Plot the time domain waveform.
ax  = fig.add_subplot(1,1,1)
ax.plot(t,m2)
ax.set_xlabel('kT [sec]')
ax.set_ylabel('m1(t)')
ax.set_xlim(tlim)

In [ ]:
df = pd.DataFrame({'time':t,'x':m2})
df = df.set_index('time')
df['env'] = df.rolling(50,center=True).max()

In [ ]:
xx = df.index
yy = df['x']
env = df['env']

tlim  = (7.3,7.6)

fig = plt.figure(figsize=(10,8)) # Create figure.

### Plot the time domain waveform.
ax  = fig.add_subplot(1,1,1)
ax.plot(xx,yy)
ax.plot(xx,env)

ax.set_xlabel('kT [sec]')
ax.set_ylabel('x(t)')
ax.set_xlim(tlim)

### Lowpass Filter

In [ ]:
wp    = 250
ws    = 1.1*wp

gpass =  3 # The maximum loss in the passband (dB).
gstop = 40 # The minimum attenuation in the stopband (dB).

N, Wn = signal.buttord(wp, ws, gpass, gstop, fs=fs)
sos   = signal.butter(N, Wn, 'low', fs=fs, output='sos')

flim = (0,150)
plot_filter_response(sos,fs,Wn,flim=flim)

In [ ]:
# Drop NaNs
df_1 = df.dropna()

In [ ]:
# Apply Filter to Signal
v1 = signal.sosfiltfilt(sos,df_1['env'])

### Lowpass Filter

In [ ]:
xx  = df.index
yy  = df['x']

tlim  = (7.3,7.6)

fig = plt.figure(figsize=(10,8)) # Create figure.

### Plot the time domain waveform.
ax  = fig.add_subplot(1,1,1)
ax.plot(xx,yy)
ax.plot(df_1.index,v1)

ax.set_xlabel('kT [sec]')
ax.set_ylabel('x(t)')
ax.set_xlim(tlim)

# High Pass Filter

In [ ]:
wp    = 10
ws    = 0.8*wp

gpass =  3 # The maximum loss in the passband (dB).
gstop = 40 # The minimum attenuation in the stopband (dB).

N, Wn = signal.buttord(wp, ws, gpass, gstop, fs=fs)
sos   = signal.butter(N, Wn, 'high', fs=fs, output='sos')

flim = (0,150)
plot_filter_response(sos,fs,Wn,flim=flim)

In [ ]:
# Apply Filter to Signal
v2 = signal.sosfiltfilt(sos,v1)

In [ ]:
xx  = df.index
yy  = df['x']

tlim  = (7.3,7.6)

fig = plt.figure(figsize=(10,8)) # Create figure.

### Plot the time domain waveform.
ax  = fig.add_subplot(1,1,1)
ax.plot(xx,yy)
ax.plot(df_1.index,v2)

ax.set_xlabel('kT [sec]')
ax.set_ylabel('x(t)')
ax.set_xlim(tlim)

# Calculate FFT

### FFT Computation
We now use the windowed version of the signal to compute the FFT.

**Important Notes**
1. For a sinusoid, we expect two delta functions with a weight of 0.5 at $\pm F_0$. To correctly calculate the amplitude we need to multiply the FFT answer by $2T_s$. Multiplying by $T_s$ is to account for the width of the sampling period. Multiplying by 2 is the correction factor for the Hanning Window that was applied earlier.
2. Remember that the Fourier Transform computes values for both positive and negative frequencies. The FFT does this, too. But, the resulting vector produced by the FFT algorithm gives the positive frequencies first followed by the negative freuqencies. To put the values in the proper order, we use np.fft.fftshift().

In [ ]:
tlim

In [ ]:
# Select only data of times that we need.



In [ ]:
# Compute the FFT.
X0 = np.fft.fft(x0_han)*Ts*2

# Reorder values so they go in frequency order from negative to positive.
X0 = np.fft.fftshift(X0)

In [ ]:
# Let's print the resulting FFT X0. Remember this should show up as complex numbers!
X0

### FFT Frequency Vector
But what are the frequencies that are actually being computed? The FFT will compute for frequencies from -Nyquist to +Nyquist, evenly spaced by the number of samples in the original signal. The function ``np.fft.fftfreq()`` will conveniently calculate the frequency vector for you in Hz, given the integer length of the input signal and the sampling period in seconds. Note that ``np.fft.fftfreq()`` matches ``np.fft.fft()``. So, we will also want to shift the order of the frequency vector.

In [ ]:
f  = np.fft.fftfreq(len(X0),Ts)
f  = np.fft.fftshift(f)

Note that you can get help about a particular command using ``pinfo``. Take a moment to read the documentation string for ``np.fft.fftfreq()``.

In [ ]:
pinfo np.fft.fftfreq

### FFT Plotting
Now that we have both the FFT values

In [ ]:
fig = plt.figure()

ax  = fig.add_subplot(2,1,1)
ax.plot(f,np.abs(X0))
ax.set_xticklabels([])
ax.set_ylabel('$|X_0(f)|$')
ax.set_title('Magnitude and Phase Spectrum of Middle C')

ax  = fig.add_subplot(2,1,2)
ax.plot(f,np.angle(X0))
ax.set_xlabel('f [Hz]')
ax.set_ylabel('arg($X_0(f)$)')
ax.set_ylim(-np.pi,np.pi)

fig.tight_layout()
plt.show()